In [1]:
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, MaxPooling2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add, LeakyReLU, Lambda
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D,Conv2DTranspose
from keras.applications import VGG19
from keras.models import Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from helper import DataLoader
import numpy as np
import os
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydot
from keras.callbacks import TensorBoard, ModelCheckpoint
from glob import glob
from tqdm import tqdm_notebook
import scipy
import imageio
from tensorlayer.prepro import *
import tensorlayer as tl
from keras import initializers
from keras import backend as K
import tensorflow as tf
import h5py

Using TensorFlow backend.


In [2]:
batch_size = 16
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0, amsgrad=False)
epochs = 2000
ni = np.sqrt(batch_size)
hr_shape = (224,224,3)
lr_shape = (56,56,3)
is_train=False

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
def Generator(lr_shape,is_train=True):
    w_init = initializers.RandomNormal(stddev=0.02, seed=None)
    g_init = initializers.RandomNormal(mean=0.0,stddev=0.02, seed=None)
    lr_input = Input(shape=lr_shape,name="in")
    n = Conv2D(64, kernel_size=9, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c')(lr_input)
    temp = n
    # Res blocks
    for i in range(16):
        nn = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c1/%s' % i)(n)
        if is_train: 
            nn = BatchNormalization(gamma_initializer=g_init, name='n64s1/b1/%s' % i)(nn)
            nn = Activation('relu')(nn)
        nn = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c2/%s' % i)(nn)
        if is_train: 
            nn = BatchNormalization(gamma_initializer=g_init, name='n64s1/b2/%s' % i)(nn)
        nn = Add(name='b_residual_add/%s' % i)([n, nn])
        n = nn

    n = Conv2D(64, (3, 3), strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c/m')(n)
    if is_train: n = BatchNormalization(gamma_initializer=g_init, name='n64s1/b/m')(n)
    n = Add(name='add3')([n, temp])
    # End of Res Blocks

    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    n = UpSampling2D(size=2)(n)
    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    if is_train: n = BatchNormalization(gamma_initializer=g_init)(n)
    n = Activation('relu')(n)

    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    n = UpSampling2D(size=2)(n)
    n = Conv2D(256, kernel_size=3, strides=1, padding='same',kernel_initializer=w_init, bias_initializer='zeros')(n)
    if is_train: n = BatchNormalization(gamma_initializer=g_init)(n)
    n = Activation('relu')(n)

    n = Conv2D(3, (9, 9),strides=1, activation='tanh', padding='same', kernel_initializer=w_init, name='out')(n)
    return Model(lr_input,n)

SRGAN_gen = Generator(lr_shape)
SRGAN_gen.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in (InputLayer)                 (None, 56, 56, 3)    0                                            
__________________________________________________________________________________________________
n64s1/c (Conv2D)                (None, 56, 56, 64)   15616       in[0][0]                         
__________________________________________________________________________________________________
n64s1/c1/0 (Conv2D)             (None, 56, 56, 64)   36928       n64s1/c[0][0]                    
__________________________________________________________________________________________________
n64s1/b1/0 (BatchNormalization) (None, 56, 56, 64)   256         n64s1/c1/0[0][0]                 
__________________________________________________________________________________________________
activation

In [4]:
def Discriminator(hr_shape,is_train=False):
    w_init = initializers.RandomNormal(stddev=0.02, seed=None)
    gamma_init = initializers.RandomNormal(mean=1,stddev=0.02, seed=None)
    inputLayer = Input(hr_shape, name='in')
    n = Conv2D(64, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s1/c')(inputLayer)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n64s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*2, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n128s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*2, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n128s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*4, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n256s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*4, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n256s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*8, kernel_size=3, strides=1, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n512s1/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Conv2D(64*8, kernel_size=3, strides=2, padding='same', kernel_initializer=w_init, bias_initializer='zeros', name='n512s2/c')(n)
    if is_train: n = BatchNormalization(gamma_initializer=gamma_init)(n)
    n = LeakyReLU(alpha=0.2)(n)
    
    n = Flatten()(n)
    n = Dense(1024)(n)
    n = LeakyReLU(alpha=0.2)(n)
    n = Dense(1,activation='sigmoid')(n)
    
    return Model(inputLayer, n)

SRGAN_disc = Discriminator(hr_shape)
SRGAN_disc.trainable = False
SRGAN_disc.compile(loss='mean_squared_error',
             optimizer=optimizer,
             metrics=['binary_accuracy'])
SRGAN_disc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (InputLayer)              (None, 224, 224, 3)       0         
_________________________________________________________________
n64s1/c (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 64)      0         
_________________________________________________________________
n64s2/c (Conv2D)             (None, 112, 112, 64)      36928     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 112, 112, 64)      0         
_________________________________________________________________
n128s1/c (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 112, 112, 128)     0         
__________

In [5]:
def buildVGG(hr_shape):
    vggInput = Input(shape=hr_shape)
    vgg = VGG19(weights="imagenet",input_tensor = vggInput)
    
    return Model(vggInput,vgg.layers[15].output)

vggModel = buildVGG(hr_shape)
for l in vggModel.layers:
    l.trainable = False

In [6]:
vggModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
img_lr = Input(shape=lr_shape)
img_hr = Input(shape=hr_shape)
gen_hr = SRGAN_gen(img_lr)
validity = SRGAN_disc(gen_hr)
vgg_features = vggModel(gen_hr)
combined = Model(img_lr, [validity, vgg_features])
combined.compile(loss=['binary_crossentropy', 'mean_squared_error'],
                              loss_weights=[1e-3, 1],
                              optimizer=optimizer)
combined.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 56, 56, 3)    0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 224, 224, 3)  3224899     input_2[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 1)            107447873   model_1[1][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 28, 28, 512)  10585152    model_1[1][0]                    
Total params: 121,257,924
Trainable params: 3,219,651
Non-trainable params: 118,038,273
_____________________

In [ ]:
SVG(model_to_dot(combined, show_layer_names=True, show_shapes=True).create(prog='dot', format='svg'))

In [7]:
path = "./mainDataset"
dataset_name = "train"
# for the VGG feature out true labels
disc_patch = (28, 28, 512)
ni = np.sqrt(batch_size)

In [8]:
train_hr_img_list = sorted(tl.files.load_file_list(path=path+'/%s/' % (dataset_name), regx='.*.png', printable=False))
train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=path+'/%s/' % (dataset_name), n_threads=32)

[TL] read 32 from ./mainDataset/train/
[TL] read 64 from ./mainDataset/train/
[TL] read 96 from ./mainDataset/train/
[TL] read 128 from ./mainDataset/train/
[TL] read 160 from ./mainDataset/train/
[TL] read 192 from ./mainDataset/train/
[TL] read 224 from ./mainDataset/train/
[TL] read 256 from ./mainDataset/train/
[TL] read 288 from ./mainDataset/train/
[TL] read 320 from ./mainDataset/train/
[TL] read 352 from ./mainDataset/train/
[TL] read 384 from ./mainDataset/train/
[TL] read 416 from ./mainDataset/train/
[TL] read 448 from ./mainDataset/train/
[TL] read 480 from ./mainDataset/train/
[TL] read 512 from ./mainDataset/train/
[TL] read 544 from ./mainDataset/train/
[TL] read 576 from ./mainDataset/train/
[TL] read 608 from ./mainDataset/train/
[TL] read 640 from ./mainDataset/train/
[TL] read 672 from ./mainDataset/train/
[TL] read 704 from ./mainDataset/train/
[TL] read 736 from ./mainDataset/train/
[TL] read 768 from ./mainDataset/train/
[TL] read 800 from ./mainDataset/train/
[TL

In [9]:
steps = len(train_hr_imgs)//batch_size

In [11]:
train_hr_imgs =  tl.prepro.threading_data(train_hr_imgs,fn=vggProcess)

In [10]:
def vggProcess(rgb):
    # Converts an image from rbg to bgr
    rgb = rgb / (255. / 2.)
    rgb = rgb - 1.
    VGG_MEAN = [103.939, 116.779, 123.68]
    red = rgb[:,:,0]
    red = np.add(red,1)
    red = red/2
    red = red * 255.0
    
    green = rgb[:,:,1]
    green = np.add(green,1)
    green = green/2
    green = green * 255.0
    
    blue = rgb[:,:,2]
    blue = np.add(blue,1)
    blue = blue/2
    blue = blue * 255.0

    bgr = np.array([
        blue - VGG_MEAN[0],
        green - VGG_MEAN[1],
        red - VGG_MEAN[2]
    ])
    bgr = np.moveaxis(bgr,0,2)
    return(bgr)
    
def datagen(dev_hr_imgs,batchSize,is_testing=False):
    while(True):
        imgs_hr=[]
        imgs_lr=[]
        imgs = np.random.choice(dev_hr_imgs,batchSize)
        img_hr = tl.prepro.threading_data(imgs, fn=crop, wrg=224, hrg=224, is_random=True)
        img_lr = tl.prepro.threading_data(img_hr, fn=imresize,size=[56, 56], interp='bicubic', mode=None)
        
        imgs_hr = np.array(img_hr)
        imgs_lr = np.array(img_lr)
        
        yield imgs_hr, imgs_lr

In [12]:
datagenObj = datagen(train_hr_imgs,batch_size)

In [13]:
sample_hr,sample_lr = next(datagenObj)

In [14]:
tl.vis.save_images(sample_hr, [int(ni), int(ni)],'images/'+dataset_name+'/sample_hr.png')
tl.vis.save_images(sample_lr, [int(ni), int(ni)],'images/'+dataset_name+'/sample_lr.png')

In [15]:
tensorboard = TensorBoard(
  log_dir='log/srgan_fixed/run6',
  histogram_freq=0,
  batch_size=batch_size,
  write_graph=True,
  write_grads=True
)
tensorboard.set_model(combined)

In [16]:
SRGAN_gen.load_weights("./checkpoints/gen.h5")
SRGAN_disc.load_weights("./checkpoints/disc.h5")

In [17]:
for epoch in range(epochs):
    print("Epoch:"+str(epoch))
    for step in tqdm_notebook(range(0,steps)):
        # Sample images and their conditioning counterparts
        imgs_hr, imgs_lr = next(datagenObj)
        # From low res. image generate high res. version
        fake_hr = SRGAN_gen.predict(imgs_lr)
        if step % 2 == 0:
            # ----------------------
            #  Train Discriminator
            # ----------------------
            
            valid = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            fake = np.random.random_sample(batch_size)*0.2

            # Train the discriminators (original images = real / generated = Fake)
            SRGAN_disc.trainable = True
            d_loss_real = SRGAN_disc.train_on_batch(imgs_hr, valid)
            d_loss_fake = SRGAN_disc.train_on_batch(fake_hr, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            SRGAN_disc.trainable = False
            
        else:
            # ------------------
            #  Train Generator
            # ------------------

            # The generators want the discriminators to label the generated images as real
            valid = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2

            # Extract ground truth image features using pre-trained VGG19 model
            image_features = vggModel.predict(imgs_hr)

            # Train the generators
            g_loss = combined.train_on_batch(imgs_lr, [valid, image_features])
        
        
    out = SRGAN_gen.predict(sample_lr)
    tl.vis.save_images(out, [int(ni), int(ni)],'images/'+dataset_name+'/train.png')
    if(epoch % 100 == 0):
        out = SRGAN_gen.predict(sample_lr)
        tl.vis.save_images(out, [int(ni), int(ni)],'images/'+dataset_name+'/train_%d.png' % int(epoch))
    tensorboard.on_epoch_end(epoch, {"d_mse": d_loss[0],"d_acc":d_loss[1],"g_loss":g_loss[0],"g_mse":g_loss[1]})
    SRGAN_gen.save_weights("./checkpoints/gen.h5")
    SRGAN_disc.save_weights("./checkpoints/disc.h5")
tensorboard.on_train_end(None)

Epoch:0


/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.



Epoch:1


...


Epoch:609



Epoch:610


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-ff3d93f13b5d>", line 7, in <module>
    fake_hr = SRGAN_gen.predict(imgs_lr)
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/keras/engine/training.py", line 1169, in predict
    steps=steps)
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 294, in predict_loop
    batch_outs = f(ins_batch)
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py", line 2675, in _call
    fetched = self._callable_fn(*array_vals)
  File "/home/ubuntu/anaconda3/envs/srgan/lib/pyt

KeyboardInterrupt: 